In [59]:
import os, sys
from pathlib import Path
import numpy as np
import math
import psutil
import operator
from functools import reduce

In [65]:
def get_size_MB(shape):
    current_size = math.prod(shape)/1024**3
    current_size *= 1
    current_size *= 1024
    return round(current_size,4)

def get_size_GB(shape, dtype=None):
    current_size = math.prod(shape)/1024**3
    current_size *=2
    
    return current_size

def optimize_chunk_shape_3d(image_shape, origional_chunks, output_chunks=(1,1024,1024)):
    
    dtype='uint16'
    cpu_cores=os.cpu_count()
    sim_jobs=4
    mem=int((psutil.virtual_memory().free/1024**3)*.8)
    chunk_limit_GB = mem / cpu_cores / 8
    y = origional_chunks[1] if origional_chunks[1] > output_chunks[1] else output_chunks[1]
    x = origional_chunks[2] if origional_chunks[2] > output_chunks[2] else output_chunks[2]

    origional_chunks = (origional_chunks[0], y, x)

    current_chunks = origional_chunks
    current_size = get_size_GB(current_chunks, dtype)

    print(current_chunks)
    print(current_size)

    if current_size > chunk_limit_GB:
        return current_chunks

    idx = 0
    chunk_bigger_than_z = True if current_chunks[0] >= image_shape[0] else False
    chunk_bigger_than_y = True if current_chunks[1] >= image_shape[1] else False
    chunk_bigger_than_x = True if current_chunks[2] >= image_shape[2] else False

    while current_size <= chunk_limit_GB:

        # last_size = get_size_GB(current_chunks,dtype)
        last_shape = current_chunks

        # chunk_iter_idx = idx % 2
        # if chunk_iter_idx == 0 and chunk_bigger_than_y == False:
        #     current_chunks = (origional_chunks[0], current_chunks[1] + output_chunks[1], current_chunks[2])
        # elif chunk_iter_idx == 1 and chunk_bigger_than_x == False:
        #     current_chunks = (origional_chunks[0], current_chunks[1], current_chunks[2] + output_chunks[2])

        # Iterate over y first then x
        if chunk_bigger_than_y == False:
            current_chunks = (origional_chunks[0],current_chunks[1]+output_chunks[1],current_chunks[2])
        elif chunk_bigger_than_x == False:
            current_chunks = (origional_chunks[0],current_chunks[1],current_chunks[2]+output_chunks[2])
        elif chunk_bigger_than_z == False:
            current_chunks = (origional_chunks[0] + output_chunks[0], current_chunks[1], current_chunks[2])

        current_size = get_size_GB(current_chunks, dtype)

        print(current_chunks)
        print(current_size)
        print('next step chunk limit {}'.format(current_size))

        if current_size > chunk_limit_GB:
            return last_shape

        if current_chunks[0] > image_shape[0]:
            chunk_bigger_than_z = True

        if current_chunks[1] > image_shape[1]:
            chunk_bigger_than_y = True

        if current_chunks[2] > image_shape[2]:
            chunk_bigger_than_x = True

        if all([chunk_bigger_than_z, chunk_bigger_than_y, chunk_bigger_than_x]):
            return last_shape

        idx += 1

        
def binlist(n, width=0):
    """Return list of bits that represent a non-negative integer.

    n      -- non-negative integer
    width  -- number of bits in returned zero-filled list (default 0)
    """
    return map(int, list(bin(n)[2:].zfill(width)))

def numVals(shape):
    """Return number of values in chunk of specified shape, given by a list of dimension lengths.

    shape -- list of variable dimension sizes"""
    print('shape', shape)
    #if(len(shape) == 0):
    #    return 1
    return reduce(operator.mul, shape)

def perturbShape(shape, onbits):
    """Return shape perturbed by adding 1 to elements corresponding to 1 bits in onbits

    shape  -- list of variable dimension sizes
    onbits -- non-negative integer less than 2**len(shape)
    """
    return map(sum, zip(shape, binlist(onbits, len(shape))))        

def chunk_shape_3D(varShape, valSize=4, chunkSize=4096):
    """
    Return a 'good shape' for a 3D variable, assuming balanced 1D/(n-1)D access

    varShape  -- length 3 list of variable dimension sizes
    chunkSize -- maximum chunksize desired, in bytes (default 4096)
    valSize   -- size of each data value, in bytes (default 4)

    Returns integer chunk lengths of a chunk shape that provides
    balanced access of 1D subsets and 2D subsets of a netCDF or HDF5
    variable var with shape (T, X, Y), where the 1D subsets are of the
    form var[:,x,y] and the 2D slices are of the form var[t,:,:],
    typically 1D time series and 2D spatial slices.  'Good shape' for
    chunks means that the number of chunks accessed to read either
    kind of 1D or 2D subset is approximately equal, and the size of
    each chunk (uncompressed) is no more than chunkSize, which is
    often a disk block size.
    """

    rank = 3
    chunkVals = chunkSize / float(valSize) # ideal number of values in a chunk
    numChunks  = varShape[0]*varShape[1]*varShape[2] / chunkVals # ideal number of chunks
    axisChunks = numChunks ** 0.25 # ideal number of chunks along each 2D axis
    cFloor = [] # will be first estimate of good chunk shape
    # cFloor  = [varShape[0] // axisChunks**2, varShape[1] // axisChunks, varShape[2] // axisChunks]
    # except that each chunk shape dimension must be at least 1
    # chunkDim = max(1.0, varShape[0] // axisChunks**2)
    if varShape[0] / axisChunks**2 < 1.0:
        chunkDim = 1.0
        axisChunks = axisChunks / math.sqrt(varShape[0]/axisChunks**2)
    else:
        chunkDim = varShape[0] // axisChunks**2
    cFloor.append(chunkDim)
    prod = 1.0  # factor to increase other dims if some must be increased to 1.0
    for i in range(1, rank):
        if varShape[i] / axisChunks < 1.0:
            prod *= axisChunks / varShape[i]
    for i in range(1, rank):
        if varShape[i] / axisChunks < 1.0:
            chunkDim = 1.0
        else:
            chunkDim = (prod*varShape[i]) // axisChunks
        cFloor.append(chunkDim)

    # cFloor is typically too small, (numVals(cFloor) < chunkSize)
    # Adding 1 to each shape dim results in chunks that are too large,
    # (numVals(cCeil) > chunkSize).  Want to just add 1 to some of the
    # axes to get as close as possible to chunkSize without exceeding
    # it.  Here we use brute force, compute numVals(cCand) for all
    # 2**rank candidates and return the one closest to chunkSize
    # without exceeding it.
    bestChunkSize = 0
    cBest = cFloor
    for i in range(8):
        # cCand = map(sum,zip(cFloor, binlist(i, rank)))
        cCand = perturbShape(cFloor, i)
        thisChunkSize = valSize * numVals(cCand)
        if bestChunkSize < thisChunkSize <= chunkSize:
            bestChunkSize = thisChunkSize
            cBest = list(cCand) # make a copy of best candidate so far
    return map(int, cBest)


In [49]:
shape = (138, 854, 568)
print(f'{shape} size {get_size_GB(shape)}GB')
shape = (3, 36962//4, 43442//4)
print(f'{shape} size {get_size_GB(shape)}GB')

(138, 854, 568) size 0.12468534708023071GB
(3, 9240, 10860) size 0.5607292056083679GB


In [68]:
#optimum_chunks = 
#utils.optimize_chunk_shape_3d_2(
#test_image.shape,
#test_image.chunks,
#self.origionalChunkSize[2:],test_image.dtype,self.res0_chunk_limit_GB)
   

image_shape = (4750, 36962, 43442)
original_chunks = (1, 1024, 1024)
image_chunks=(250, 1536, 1024)
#chunks = optimize_chunk_shape_3d(image_shape, image_chunks)
chunks = chunk_shape_3D(image_shape)
print(list(chunks))

shape <map object at 0x7ebda659fa60>
shape <map object at 0x7ebda659f580>
shape <map object at 0x7ebda659eb30>
shape <map object at 0x7ebda659fa90>
shape <map object at 0x7ebda659fa60>
shape <map object at 0x7ebda659f580>
shape <map object at 0x7ebda659eb30>
shape <map object at 0x7ebda659fa90>
[]
